In [12]:
import os
import tempfile
import anndata
import scvi
import subprocess
import pandas as pd
from scib_metrics.benchmark import Benchmarker

# 获取 Git 仓库的根目录
git_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"]).strip().decode('utf-8')

# 更改当前工作目录到 Git 仓库的根目录
os.chdir(git_root)

In [ ]:
os.getcwd()
adata = anndata.read_h5ad("data/201.load_sc/sc_pre.h5ad")
# 将adata的X矩阵复制到adata的layers["counts"]中，并转换为CSR格式
adata.layers["counts"] = adata.X.copy().tocsr()

# 运行setup_anndata，使用列"dataset"作为批次键
scvi.model.SCVI.setup_anndata(adata, batch_key="batch", layer="counts")

# 创建模型
model = scvi.model.SCVI(adata, n_layers=2, n_latent=30, gene_likelihood="nb")

# 训练模型
model.train(accelerator="cpu")

# 获取潜在表示
latent = model.get_latent_representation()

# 假设 latent 是一个 numpy.ndarray
latent_df = pd.DataFrame(latent)

# 将 DataFrame 保存为 TSV 文件
latent_df.to_csv("data/201.load_sc/scvi_latent.tsv", sep="\t", index=True, header=True)